In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
ac,ao,hc,ho = 0,1,2,3
path_data ="/Users/sanaailla/Desktop/Intern/Data/mars/"
patients = pd.read_csv(path_data + "dir_list.csv",header=None)
saturations = pd.read_csv(path_data + "Saturations_all.csv", index_col = None)
desaturations = pd.read_csv(path_data + "Desaturations_all.csv", index_col = None)
hypervs = pd.read_csv(path_data + "Hypervs_all.csv", index_col = None)
annotations = pd.read_csv(path_data + "Annotations_all.csv",index_col = None)
desaturations.head()

,MAX_desat,MIN_desat,MAX_resat,HEURE_debut,HEURE_min,HEURE_fin,DUREE_event,DUREE_desat,DUREE_resat,DELTA_desat,DELTA_resat,PENTE_desat,PENTE_resat,patient
0,98.498497,95.498543,99.498482,85721.312500,85748.296875,85750.968750,29.656250,26.984375,2.671875,3.0,3.999939,-0.111175,1.497053,PA1011
1,95.498543,92.498589,95.498543,85863.296875,85891.640625,85917.296875,54.000000,28.343750,25.656250,3.0,2.999954,-0.105843,0.116929,PA1011
2,96.498528,93.498573,96.498528,85921.296875,85922.296875,86016.968750,95.671875,1.000000,94.671875,3.0,2.999954,-3.000000,0.031688,PA1011
3,96.498528,92.498589,97.498512,86016.968750,86035.296875,86064.625000,47.656250,18.328125,29.328125,4.0,4.999924,-0.218244,0.170482,PA1011
4,97.498512,93.498573,96.498528,86064.625000,86072.656250,86080.625000,16.000000,8.031250,7.968750,4.0,2.999954,-0.498054,0.376465,PA1011


In [ ]:
patient = patients[0][0]
hyp = hypervs[hypervs["patient"]==patient]
sat = saturations[saturations["patient"] == patient]
m = sat.SAT[0]
for i in range(len(hyp)):
    hypo = sat[(sat.Heure >= hyp.HEURE_debut[i]) & (sat.Heure <= hyp.HEURE_fin[i])].SAT
    plt.plot(hypo)
    #plt.plot(hypo[hypo>=m],c = "green")
    plt.axhline(m,c="red")
    plt.plot()
    plt.show()

In [ ]:
def desaturations_lab(desaturations, saturations,patients, seuil = 90) :
    toreturn = pd.DataFrame(columns = desaturations.columns)
    toreturn["events"] = np.NAN
    for patient in patients:
        desat = desaturations[desaturations["patient"] == patient].reset_index()
        sats = saturations[saturations["patient"] == patient].reset_index()
        data = pd.DataFrame(data = desat,columns = desaturations.columns)
        debut_desat = desat["HEURE_debut"].astype(int) + 1
        debut_label = sats.Heure
        events = sats["Évenement"]
        duree_label = sats.Durée
        labels = []
        closest_times = []
        differences = []
        for i in range(len(desat)):
            closest_label = np.NaN
            closest_time = np.NaN
            #diff = np.NaN
            for label_time,event,duree in zip(debut_label,events,duree_label):
                if label_time < debut_desat[i] and (debut_desat[i] - label_time) < seuil + duree  and label_time not in closest_times:
                    closest_label = event
                    closest_time = label_time
                    #diff =  debut_desat[i] - label_time
            #differences.append(diff)
            labels.append(closest_label)
            closest_times.append(closest_time)
        data["events"] = labels
        #desaturations["difference"] = differences
        #desaturations["closest time"] = closest_times
        data = data[~data["events"].isna()].reset_index(drop=True)
        data["events"] =  data["events"].replace({"hypopnée Centrale": hc,"Hypopnée": ho,"Apnée Obstructive": ao,"Apnée Centrale": ac})
        toreturn = pd.concat([data,toreturn])
        print("fin",patient)
    return toreturn

In [ ]:
desaturations_labelise = desaturations_lab(desaturations,saturations,patients[0])
desaturations_labelise.to_csv(path_data + "Desaturations_all_labelisee.csv")

In [ ]:
desaturations_labelisee = pd.read_csv(path_data + "Desaturations_all_labelisee.csv",index_col= 0)
desaturations_labelise.columns

In [ ]:
import numpy as np
from scipy.integrate import trapz
def attribute_hypervs(desaturations,saturations,hypervs,patients,ecart = 0):
    toreturn = pd.DataFrame()
    for patient in patients:
        desat = desaturations[desaturations["patient"] == patient].reset_index()
        sats = saturations[saturations["patient"] == patient].reset_index()
        hyp = hypervs[hypervs["patient"] == patient].reset_index()
        m = sats.SAT[0]
        i = 0
        ixs = []
        desat["Fin hyp"] = 0
        desat["Hyp"] = 0
        desat["area_up"] = 0
        while i < len(desat):
            seuil = int(desat.DUREE_event[i]) + ecart
            j = 0
            while j < len(hyp):
                dff_time = hyp.HEURE_debut[j] - desat.HEURE_debut[i]
                if dff_time >= 0 and dff_time <= seuil:
                    ixs.append((i,j))
                    break
                j+= 1
            desaturation = sats[((sats.Heure >= desat.HEURE_debut[i]) & (sats.Heure <= desat.HEURE_fin[i]))].SAT
            x = desaturation.index.values
            y = desaturation.values
            baseline = [m for _ in range(len(desaturation))]
            desat.loc[i, "area_down"] = trapz(np.maximum(y, baseline) - baseline, x)
            i+= 1
        for (d,h) in ixs:
            signal_hyp = sats[(sats.Heure >= hyp.HEURE_min[h]) & (sats.Heure <= hyp.HEURE_fin[h])].SAT
            indice_sig = (signal_hyp >= m)
            indice_descente = indice_sig[indice_sig == False].index
            if (len(indice_descente)!=0):
                indice_descente = indice_descente.values[0]
                heure_fin = sats.iloc[indice_descente].Heure
            else :
                heure_fin = hyp.HEURE_fin[h]
            desaturation = sats[(sats.Heure >= desat.HEURE_debut[d]) & (sats.Heure <= heure_fin)].SAT
            if desaturation.values[-1] <m:
                desaturation = desaturation[:-1]
            x = desaturation.index.values
            y = desaturation.values
            reference_line = [m for _ in range(len(desaturation))]
            desat.loc[d, "area_up"] = trapz(np.maximum(y, reference_line) - reference_line, x)
            #desat.loc[d, "area_down"] = trapz(np.maximum(y, reference_line) - reference_line, x)
            desat.loc[d,"Fin hyp"] = heure_fin
            desat.loc[d,"Hyp"] = 1
            #plt.plot(desaturation)
            #plt.axhline(m,c="red")
            #plt.show()
        toreturn = pd.concat([desat,toreturn])
        print("fin", patient)
    return toreturn

In [ ]:
final_desats = attribute_hypervs(desaturations_labelise,saturations,hypervs,patients[0],0)

In [14]:
final_desats.to_csv(path_data + "final_desats.csv")

In [10]:
final_desats[final_desats.Hyp == 1]

In [ ]:
def display_desaturations(desaturations,saturations):
    for i in range(len(desaturations)):
        heure_fin = desaturations.HEURE_fin[i]
        if desaturations["Fin hyp"][i]!=0:
            heure_fin = desaturations["Fin hyp"][i]
            desaturation = saturations[(saturations.Heure >= desaturations.HEURE_debut[i]) & (saturations.Heure <= heure_fin)].SAT
            plt.plot(desaturation)
            m = saturations.SAT[0]
            plt.axhline(m,c="red")
            plt.show()

In [ ]:
for patient in patients[0]:
    display_desaturations((final_desats[final_desats["patient"] == patients[0][0]].reset_index()),(saturations[saturations["patient"] == patients[0][0]].reset_index()))